In [ ]:
%matplotlib inline
# imports
import os
import sys
sys.path.insert(0, os.path.abspath(r'D:\Code Repos\prey_capture'))
import functions_bondjango as bd
import paths
import os
from caiman.base.rois import register_multisession

import h5py
import numpy as np
import functions_misc as fm
import processing_parameters
import sys
import re
import json
import functions_plotting as fplot
import processing_parameters
import matplotlib.pyplot as plt
from matplotlib import cm

In [ ]:
# load the data

# define the target files
# get the search string
animal = processing_parameters.animal
rig = processing_parameters.rig
search_string = processing_parameters.search_string + 'analysis_type:calciumday'
# query the database for data to plot
data_all = bd.query_database('analyzed_data', search_string)
# get the paths to the files
calcium_path = [el['analysis_path'] for el in data_all]
# # for testing, filter calcium path
# calcium_path = [el for el in calcium_path if ('03_24' in el) or ('03_23' in el) or ('03_29' in el)]

# load the data for the matching
footprint_list = []
size_list = []
template_list = []
# # also store the frame lists
# frame_lists = []
# also store the date for each file
date_list = []
# load the calcium data
for files in calcium_path:
    with h5py.File(files, mode='r') as f:
        try:
            calcium_data = np.array(f['A'])
        except KeyError:
            continue

        # if there are no ROIs, skip
        if (type(calcium_data) == np.ndarray) and (calcium_data == 'no_ROIs'):
            continue
        footprint_list.append(np.moveaxis(calcium_data, 0, -1).reshape((-1, calcium_data.shape[0])))
        size_list.append(calcium_data.shape[1:])
        template_list.append(np.zeros(size_list[0]))
        # template_list.append(np.array(f['max_proj']))
        date_list.append(os.path.basename(files)[:10])
        # frame_lists.append(np.array(f['frame_list']))
print(f'Number of days selected: {len(footprint_list)}')

In [ ]:
# run cell matching
# original thres = 1.1, then 0.5, 0.7, 0.9, 1.0 from more to less stringent, 1.0 still holds
# run the matching software
spatial_union, assignments, matchings = register_multisession(
    A=footprint_list, dims=size_list[0], templates=template_list, thresh_cost=0.9)

In [ ]:
# visualize cell matching

print(f'Number of total cells: {assignments.shape[0]}')
print(f'Number of total days: {assignments.shape[1]}')

# idea is one defines the target trials to visualize
# then, only the cells matched in all those are selected
# then each cell is plotted in a different color in the same plot (color is session)
# at the end, all unmatched cells are plotted

# define the sessions to use
target_sessions = [0, 1, 2]
# get the selected days
target_matchings = assignments[:, target_sessions]
# get only the cells with all matchings
all_matched = target_matchings[np.sum(np.isnan(target_matchings), axis=1)==0, :]

# get the number of cells
cell_number = np.min([30, all_matched.shape[0]])
# generate the colors for all the cells
cell_colors = cm.jet(np.arange(len(target_sessions))*1/(len(target_sessions)-1))
# generate the figure array
nrows = int(np.round(np.sqrt(cell_number)))
ncols = int(np.ceil(np.sqrt(cell_number)))
# ncols = 2
# nrows = int(np.ceil(cell_number/2))
fig1, ax_array = plt.subplots(figsize=(200, 200), nrows=nrows, ncols=ncols)

# for all the units
for cell in np.arange(cell_number):
    # allocate a list for the masks
    cell_masks = []
    
    # for all the trials
    for trial in np.arange(all_matched.shape[1]):
        # get the corresponding cell index
        current_cell = int(all_matched[cell, trial])
        # get the current mask
        current_mask = footprint_list[target_sessions[trial]][:, current_cell]
        # normalize between 0 and 1
        current_mask[current_mask > 0] = 1
        # reshape and color
        current_mask = np.tile(current_mask.reshape([630, 630]), [3, 1, 1])
        current_mask[0, :, :] = current_mask[0, :, :]*cell_colors[trial, 0]
        current_mask[1, :, :] = current_mask[1, :, :]*cell_colors[trial, 1]
        current_mask[2, :, :] = current_mask[2, :, :]*cell_colors[trial, 2]
                
        # store
        cell_masks.append(current_mask)
    # combine days
    cell_masks = np.transpose(np.sum(cell_masks, axis=0), [1, 2, 0])
#     cell_masks = cell_masks[0].reshape([630, 630])
    # generate plot
    idx0, idx1 = np.unravel_index(cell, [nrows, ncols])
    plt.sca(ax_array[idx0][idx1])
    plt.imshow(cell_masks)
